<a href="https://colab.research.google.com/github/SushmaMacha/SkinLesions/blob/master/transfer_learning_finetune_wBottleneck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s gdrive/'My Drive'/SJSU/'machine learning'/'CMPE257 Fall2018 Team Project'/Code/data data
!ln -s gdrive/'My Drive'/SJSU/'machine learning'/'CMPE257 Fall2018 Team Project'/Code code

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import numpy as np
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, Model
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Input
from keras import backend as K

# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/val'
nb_train_samples = 2*800
nb_validation_samples = 2*192 #batch_size multiple
epochs = 50
batch_size = 16

Using TensorFlow backend.


In [0]:
datagen = ImageDataGenerator(rescale=1. / 255)

# build the VGG16 network
model = applications.VGG16(include_top=False, weights='imagenet')
weights = model.get_weights()
config = model.get_config()
model = Sequential.from_config(config)
model.set_weights(weights)
model.pop()
model.pop()
model.pop()
model.pop()
model.summary()

generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)
bottleneck_features_train = model.predict_generator(
    generator, nb_train_samples // batch_size,
    use_multiprocessing=True)
np.save(open('data/bottleneck_features_train_ft_3lay.npy', 'wb'),
        bottleneck_features_train)

generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)
bottleneck_features_validation = model.predict_generator(
    generator, nb_validation_samples // batch_size,
    use_multiprocessing=True)
np.save(open('data/bottleneck_features_validation_ft_3lay.npy', 'wb'),
        bottleneck_features_validation)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, None, None, 256)   295168    
__________

In [0]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.utils.np_utils import to_categorical  

top_model_weights_path = 'data/bottleneck_fc_model.h5'
# build the VGG16 network
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
vgg = applications.VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
weights = vgg.get_weights()
config = vgg.get_config()
vgg = Sequential.from_config(config)
vgg.set_weights(weights)

#Prep data and labels
datagen_top = ImageDataGenerator(rescale=1./255)  
generator_top = datagen_top.flow_from_directory(  
       train_data_dir,  
       target_size=(img_width, img_height),  
       batch_size=batch_size,  
       class_mode='categorical',  
       shuffle=False)

nb_train_samples = len(generator_top.filenames)
num_classes = len(generator_top.class_indices)

train_data = np.load(open('data/bottleneck_features_train_ft_3lay.npy', 'rb'))

train_labels = np.array(
    [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))


# get the class lebels for the training data, in the original order
#train_labels = generator_top.classes  
# convert the training labels to categorical vectors  
#train_labels = to_categorical(train_labels, num_classes=2)  

generator_top = datagen_top.flow_from_directory(  
       validation_data_dir,  
       target_size=(img_width, img_height),  
       batch_size=batch_size,  
       class_mode=None,  
       shuffle=False)  

nb_validation_samples = len(generator_top.filenames)  

validation_data = np.load(open('data/bottleneck_features_validation_ft_3lay.npy', 'rb'))

#validation_labels = generator_top.classes  
#validation_labels = to_categorical(validation_labels, num_classes=num_classes)  

validation_labels = np.array(
    [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

#adding the last conv block from vgg and one hidden layer on top of that:
input2 = Input(shape=train_data.shape[1:])
x = vgg.get_layer("block5_conv1")(input2)
x = vgg.get_layer("block5_conv2")(x)
x = vgg.get_layer("block5_conv3")(x)
x = vgg.get_layer("block5_pool")(x)
x = Flatten()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
preds = Dense(1, activation="sigmoid")(x)
model = Model(input2, preds)

for layer in ["block5_conv1", "block5_conv2", "block5_conv3"]:
  weights = vgg.get_layer(layer).get_weights()
  model.get_layer(layer).set_weights(weights)

model.load_weights(top_model_weights_path, by_name=True)

#Can see better val_acc if trainable is not False
#for layer in model.layers[:2]:
#    layer.trainable = False

model.summary()

#Train top model
#model.compile(loss='binary_crossentropy',
#              optimizer=optimizers.SGD(lr=1e-3, momentum=0.9),
#              metrics=['accuracy'])
#model.compile(optimizer='rmsprop',
#              loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer=optimizers.Adam(),
             loss='binary_crossentropy',
             metrics=['accuracy'])

# Save the model according to the conditions  
checkpoint = ModelCheckpoint('data/fine_tune_bn.h5', monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
tensorboard = TensorBoard(log_dir='./data/tensorboard_logs', histogram_freq=0, batch_size=batch_size, write_graph=True, write_grads=True)

model.fit(train_data, train_labels,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels),
          callbacks = [checkpoint, early, tensorboard])

Found 1600 images belonging to 2 classes.
Found 384 images belonging to 2 classes.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 9, 9, 512)         0         
_________________________________________________________________
block5_conv1 (Conv2D)        (None, 9, 9, 512)         2359808   
_________________________________________________________________
block5_conv2 (Conv2D)        (None, 9, 9, 512)         2359808   
_________________________________________________________________
block5_conv3 (Conv2D)        (None, 9, 9, 512)         2359808   
_________________________________________________________________
block5_pool (MaxPooling2D)   (None, 4, 4, 512)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)              0         
___________________________________________________________